# Analysis - Movie Subset of Netflix Data

Enhanced by Rotten Tomatoes Dataset

#### Import Libraries

In [2]:
import pandas as pd
import plotly.express as px

Create DataFrame from Netflix CSV, filtered to just movies

Create DataFrame from Rotten Tomatoes Movies CSV

In [3]:
netflix_movies_df = pd.read_csv('./data/netflix-titles.csv')
netflix_movies_df = netflix_movies_df[netflix_movies_df["type"] == 'Movie']
rotten_movies_df = pd.read_csv('./data/rotten_tomatoes_movies.csv')

Extract release_year from Rotten Tomatoes original_release_date

In [4]:
rotten_movies_df["release_year"] = rotten_movies_df["original_release_date"].str.split('-').str[0]

Add Unique identifier column to both DataFrames for merging

In [5]:
def combine_title_and_year_columns(dataframe, title_column, year_column, output_column):
  dataframe[output_column] = dataframe[title_column].str.lower() + " " + netflix_movies_df[year_column].astype(str)

netflix_movies_df["title_and_release_year"] = netflix_movies_df["title"].str.lower() + " " + netflix_movies_df["release_year"].astype(str)
# combine_columns(netflix_movies_df, "title", "release_year", output_column="title_and_release_year")
rotten_movies_df["title_and_release_year"] = rotten_movies_df["movie_title"].str.lower() + " " + rotten_movies_df["release_year"].astype(str)
# combine_columns(rotten_movies_df, "movie_title", "release_year", output_column="title_and_release_year")

Merge the above datasets, returning only those that exist in both sets

In [6]:
netflix_plus_rt_df = pd.merge(netflix_movies_df, rotten_movies_df, how="inner", on="title_and_release_year")
after_manual_join = netflix_plus_rt_df["title"].count()
netflix_plus_rt_df.describe()

,release_year_x,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,1252.000000,1232.000000,1251.000000,1251.000000,1248.000000,1.248000e+03,1252.000000,1252.000000,1252.000000
mean,2008.528754,107.604708,59.055156,102.459632,61.138622,4.664194e+05,24.381789,64.581470,37.818690
std,11.077374,20.597857,28.775451,85.067835,20.222044,3.310599e+06,17.706812,71.982482,39.504686
min,1954.000000,40.000000,0.000000,5.000000,8.000000,6.000000e+00,0.000000,0.000000,0.000000
25%,2004.000000,93.000000,35.000000,30.500000,46.000000,1.461500e+03,8.000000,13.000000,7.000000
50%,2012.000000,103.000000,63.000000,80.000000,63.000000,2.680450e+04,21.000000,39.000000,22.000000
75%,2017.000000,118.000000,85.500000,158.000000,77.250000,1.685648e+05,39.000000,89.000000,59.000000
max,2020.000000,213.000000,100.000000,512.000000,99.000000,3.467977e+07,65.000000,495.000000,196.000000


# Histogram - Netflix Catalogue of Movies by Release Year

Stacked by Rotten Tomatoes Tomatometer status
- ranges from Rotten to Certified Fresh
- [Rotten Tomatoes Explained](rottentomatoes.com/about)

In [7]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status") # Remove those without a tomatometer status
netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["release_year_x"] >= 2000] # Remove Movies released before 2000

netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)

fig = px.histogram(
  netflix_plus_rt_df,
  "release_year_x",
  title="Movies Per Year with Rotten Tomatoes Rating [2000-2020]",
  labels={"release_year_x": "Release Year"},
  color="tomatometer_status",
  color_discrete_map={
                "Rotten": "hsl(6, 78%, 66%)",
                "Fresh": "hsl(48, 89%, 60%)",
                "Certified-Fresh": "hsl(145, 61%, 59%)",
                },
)

fig.update_layout(
  barmode="group",
  xaxis_tickangle=-45,
  width=1000,
  )

In [8]:
# netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status") # Remove those without a tomatometer status
# netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["release_year_x"] >= 2000] # Remove Movies released before 2000
# is_any_fresh_filt = (netflix_plus_rt_df["tomatometer_status"] == "Certified-Fresh") | (netflix_plus_rt_df["tomatometer_status"] == "Fresh")

# netflix_plus_rt_df["Is Fresh"] = netflix_plus_rt_df[is_any_fresh_filt]

# netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)


# fig = px.histogram(
#   netflix_plus_rt_df,
#   "release_year_x",
#   title="Movies Per Year with Rotten Tomatoes Rating [2000-2020]",
#   labels={"release_year_x": "Release Year"},
#   color="Is Fresh",
#   color_discrete_map={
#                 False: "hsl(6, 78%, 66%)",
#                 True: "hsl(145, 61%, 59%)",
#                 },
# )

# fig.update_layout(
#   barmode="group",
#   xaxis_tickangle=-45
#   )


In [9]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status")
# px.histogram(netflix_plus_rt_df, "release_year_x", color="tomatometer_status", labels={"release_year_x": "Release Year"}, color_discrete_sequence=['red', 'blue', 'green'], title="Movies Per Year with Rotten Tomatoes Rating")
netflix_plus_rt_df = netflix_plus_rt_df.sort_values(by=["tomatometer_status", "release_year_x"], ascending=False)

certified_fresh_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Certified-Fresh"]
fresh_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Fresh"]
rotten_netflix_plus_rt_df = netflix_plus_rt_df[netflix_plus_rt_df["tomatometer_status"] == "Rotten"]

px.histogram(
  netflix_plus_rt_df,
  "release_year_x",
  barmode="group",
  title="Movies Per Year rated Rotten",
  width=1000,
  labels={"release_year_x": "Release Year"},
)

In [10]:
netflix_plus_rt_df = netflix_plus_rt_df.dropna(subset="tomatometer_status")
df_grouped = netflix_plus_rt_df.groupby("release_year_x").count().reset_index()
px.bar(df_grouped,
      x='release_year_x',
      y='title',
      title='Test',
      color='tomatometer_status',
      barmode="stack",
      color_discrete_map={
        "Rotten": "red",
        "Fresh": "yellow",
        "Certified-Fresh": "green",
        },
      width=1000,
      labels={
        "release_year_x": "Release Year",
        "title": "Count",
        }
      )

  ### Plan for Wednesday 15/06/2022
  - Refactor into functions [Timebox: 1hr => 11:30]
  - Python-Slugify & Peter Norvig spelling => pre-merge
  - **5** different types of graph

### Exploring Slugify

In [11]:
from slugify import slugify

entry_nf = 330 # Index of Cloudy with a chance of meatballs in Netflix
entry_rt = 1674 # Index of Cloudy with a chance of meatballs in Rotten Tomatoes

# film = netflix_movies_df.loc[netflix_movies_df['title'].str.lower() == 'Cloudy with a chance of meatballs'.lower()]["title"]
# print(film)

netflix_title = netflix_movies_df.loc[entry_nf]["title"]
netflix_year = netflix_movies_df.loc[entry_nf]["release_year"]
rotten_title = rotten_movies_df.loc[entry_rt]["movie_title"]
rotten_year = rotten_movies_df.loc[entry_rt]["release_year"]

nf_title_slug = slugify(f"{netflix_title} {netflix_year}")
rt_title_slug = slugify(f"{rotten_title} {rotten_year}")

print(f"{netflix_title} ({netflix_year}) => {nf_title_slug}")
print(f"{rotten_title} ({rotten_year}) => {rt_title_slug}")
print(f"slugs match: {nf_title_slug == rt_title_slug}")


Cloudy with a Chance of Meatballs (2009) => cloudy-with-a-chance-of-meatballs-2009
Cloudy With a Chance of Meatballs (2009) => cloudy-with-a-chance-of-meatballs-2009
slugs match: True


In [12]:
def combine_title_and_year_columns(dataframe, title_column, year_column, output_column):
  dataframe[output_column] = dataframe[title_column].str.lower() + " " + netflix_movies_df[year_column].astype(str)

# netflix_movies_df["slug"] = [slugify(f"{entry['title']} {entry['release_year']}") for entry in netflix_movies_df]
# rotten_movies_df["slug"] = slugify(f"{rotten_movies_df['movie_title']} {rotten_movies_df['release_year']}")

netflix_movies_df["title_and_release_year"] = netflix_movies_df["title"] + " " + netflix_movies_df["release_year"].astype(str)
netflix_movies_df["title_and_release_year"] = netflix_movies_df["title_and_release_year"].apply(slugify)
# combine_columns(netflix_movies_df, "title", "release_year", output_column="title_and_release_year")
rotten_movies_df["title_and_release_year"] = rotten_movies_df["movie_title"] + " " + rotten_movies_df["release_year"].astype(str)
rotten_movies_df["title_and_release_year"] = rotten_movies_df["title_and_release_year"].apply(slugify)
# combine_columns(rotten_movies_df, "movie_title", "release_year", output_column="title_and_release_year")


netflix_plus_rt_df = pd.merge(netflix_movies_df, rotten_movies_df, how="inner", on="title_and_release_year")
after_slugify = netflix_plus_rt_df["title"].count()
netflix_plus_rt_df.describe()


,release_year_x,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,1283.000000,1261.000000,1280.000000,1280.000000,1278.000000,1.278000e+03,1283.000000,1283.000000,1283.000000
mean,2008.474669,107.389374,58.914844,101.823437,61.065728,4.579191e+05,24.239283,63.951676,37.678878
std,11.035326,20.698359,28.763354,84.786842,20.171009,3.272016e+06,17.674328,71.532778,39.492112
min,1954.000000,22.000000,0.000000,5.000000,8.000000,6.000000e+00,0.000000,0.000000,0.000000
25%,2004.000000,93.000000,35.000000,30.000000,46.000000,1.481250e+03,8.000000,13.000000,7.000000
50%,2012.000000,103.000000,62.500000,79.000000,63.000000,2.709750e+04,21.000000,38.000000,21.000000
75%,2017.000000,118.000000,85.000000,158.000000,77.000000,1.707082e+05,39.000000,87.500000,59.000000
max,2020.000000,213.000000,100.000000,512.000000,99.000000,3.467977e+07,65.000000,495.000000,196.000000


### Create a dataframe containing the results

In [25]:
complete_nf_rt_df = pd.merge(netflix_movies_df, rotten_movies_df, how="outer", left_on="title", right_on="movie_title", indicator=True)
# complete_nf_rt_df = pd.merge(netflix_movies_df, rotten_movies_df, how="outer", on="title_and_release_year", indicator=True)
# unlinked_netflix_plus_rt_df = netflix_plus_rt_df.drop(list(netflix_plus_rt_df))
complete_nf_rt_df["title"].fillna(complete_nf_rt_df["movie_title"])
complete_nf_rt_df["movie_title"].fillna(complete_nf_rt_df["title"])
included_in_both_filt = complete_nf_rt_df["_merge"] == "both"
complete_nf_rt_df = complete_nf_rt_df[~included_in_both_filt]
complete_nf_rt_df.describe()

,release_year_x,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,4342.000000,15579.000000,15764.000000,15764.000000,15541.000000,1.554000e+04,15805.000000,15805.000000,15805.000000
mean,2014.213266,101.873676,60.960606,53.667660,60.601570,1.166589e+05,13.856438,34.175324,19.434419
std,8.994796,18.558062,28.429809,65.699715,20.548184,1.547059e+06,14.680692,50.250230,29.139070
min,1942.000000,5.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000
25%,2014.000000,90.000000,39.000000,11.000000,45.000000,7.200000e+02,3.000000,6.000000,3.000000
50%,2017.000000,99.000000,67.000000,26.000000,63.000000,4.060000e+03,8.000000,15.000000,7.000000
75%,2019.000000,110.000000,86.000000,68.000000,78.000000,2.125700e+04,21.000000,41.000000,22.000000
max,2021.000000,266.000000,100.000000,554.000000,100.000000,3.579764e+07,69.000000,497.000000,303.000000


## Highlight the Progress on Cleaning Pre-Merge

In [59]:
incredible_progress = {
  'stage': ["after_manual_join", "after_slugify"],
  'count': [after_manual_join, after_slugify],
}
incredible_progress_df = pd.DataFrame.from_dict(incredible_progress)
incredible_progress_df

px.line(
  incredible_progress_df['count'],
  width=1000,
)

px.bar(
  incredible_progress_df['count'],
  width=1000,
  log_y=True
)

# px.bar(
#   incredible_progress_df['count'],
#   width=1000,
# )